# Asymptotic expansion of the energy {#sec-20230523130742}

This notebook uses $\LaTeX\newcommand{\D}{\mathrm{d}}\newcommand{\E}{\mathcal{E}}\newcommand{\order}[2][1]{#2^{(#1)}}\newcommand{\reals}{\mathbb{R}}$ custom macros.

In this chapter, we derive the asymptotic expansion (@eq-20230528190539) of the energy along the bifurcated branch. More precisely, the following quantity is expanded at the critical point

$$
\Delta\E(\eta) = \E[u(\eta), \lambda(\eta)] - \E[u^\ast \circ \lambda(\eta), \lambda(\eta)],
$$

where $\eta$ is an arbitrary parametrization of the bifurcated branch ($\eta = 0$ at the critical point).

The asymptotic expansion of the energy results from plugging the asymptotic expansions (@eq-20230528174310) and (@eq-20230411141319) of $\lambda(\eta)$ and $u(\eta)$ into the general expression of the energy that was derived in @sec-20230402140105.

In [1]:
from lsk import *
from sympy import *

We first define the asymptotic expansion of $\lambda$ w.r.t. the (unspecified) parameter $\eta$.

In [2]:
def create_λ():
    return λ1 * η + λ2 * η**2 / 2 + λ3 * η**3 / 6 + λ4 * η**4 / 24 + O(η**5)

In [3]:
#| code-fold: true
display_latex_equation(r"\lambda(\eta)", create_λ())

<IPython.core.display.Math object>

In the previous chapters, we have derived the asymptotic expansions of $\lambda$, $u^\ast$ and $u$ 
up to fourth-order in $\eta$. It will be shown below that these expansion lead to an expansion of $\Delta\E$ up to **fourth-order** terms in $\eta$.

For the sake of consistency, expansions of $\lambda$, $u^\ast$ and $u$ will be postulated up to fourth-order: it will be finally checked that the fourth-order expansion of $\Delta\E$ refers only to: $\order[1]{\lambda}$, $\order[2]{\lambda}$, $w_{i\lambda}$ and $w_{ij}$ (which are all known quantities).

The fundamental branch $u^\ast(\lambda)$ was previously defined as an asymptotic expansion w.r.t. $\lambda$. We can now define it as a function of $\eta$, keeping only fourth-order terms.

In [4]:
def create_u_star_2():
    return create_u_star(λ_fun=create_λ)

In [5]:
#| code-fold: true
display_latex_equation(r"u^\ast(\eta)", create_u_star_2().expand())

<IPython.core.display.Math object>

Similarly, we define the asymptotic expansion of $u$, see Eqs. (@eq-20230528174310) and (@eq-20230411141319); note that the quantities $\order[3]{\tilde{u}}$ and $\order[4]{\tilde{u}}$ are unknown (and will in fact disappear from the derivation). 

In [6]:
u1 = Symbol(r"{\order[1]{u}}")
u2_V = Symbol(r"{\order[2]{u}_V}")
u2_W = Symbol(r"{\order[2]{u}_W}")
u3 = Symbol(r"{\order[3]{u}}")
u4 = Symbol(r"{\order[4]{u}}")

def create_u():
    return (create_u_star(create_λ) + η * u1 + η**2 / 2 * (u2_V + u2_W) + η**3 / 6 * u3 + η**4 / 24 * u4)

In [7]:
#| code-fold: true
display_latex_equation(r"u(\eta)", create_u().expand())

<IPython.core.display.Math object>

We then use these expansions to evaluate the energy along the **fundamental** and **bifurcated** branches, as well as the difference $\Delta\E$ of these two quantities. The resulting expressions are too long to be displayed. We first apply some elementary simplifications.

In [8]:
E_fund = create_E(λ_fun=create_λ, u_fun=create_u_star_2).expand()
E_bif = create_E(λ_fun=create_λ, u_fun=create_u).expand()

In [9]:
_λ = Idx(r"\lambda")

def u1_(i1):
    return ξ1_[i1] * v_[i1]

def u2_V_(i1):
    return ξ2_[i1] * v_[i1]

def u2_W_(i1, i2):
    return ξ1_[i1] * (ξ1_[i2] * w_[i1, i2] + 2 * λ1 * w_[i1, _λ])

ΔE = expand(E_bif - E_fund).subs({
    E2 * u1: 0,
    E2 * u2_V: 0
})

my_list  = [
    (u1 ** 4, u1_(i) * u1_(j) * u1_(k) * u1_(l)),
    (u1 ** 3, u1_(i) * u1_(j) * u1_(k)),
    (u1 ** 2 * u2_V, u1_(i) * u1_(j) * u2_V_(k)),
    (u1 ** 2 * u2_W, u1_(i) * u1_(j) * u2_W_(k, l)),
    (u1 ** 2, u1_(i) * u1_(j)),
    (u2_W ** 2, u2_W_(i, j) * u2_W_(k, l)), 
    (u1 * u2_V, u1_(i) * u2_V_(j)),
    (u1 * u2_W, u1_(i) * u2_W_(j, k))
]

for pair in my_list:
    ΔE = ΔE.subs(*pair)

ΔE = ΔE.expand()
ΔE

\dot{\E}_3*\eta**4*{\order[1]{\lambda}}*v[i]*v[j]*v[k]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + \dot{\E}_2*\eta**3*{\order[1]{\lambda}}*v[i]*v[j]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]/2 + \dot{\E}_2*\eta**4*{\order[2]{\lambda}}*v[i]*v[j]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]/4 + \dot{\E}_2*\eta**4*{\order[1]{\lambda}}*v[i]*w[j, k]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/2 + \dot{\E}_2*\eta**4*{\order[1]{\lambda}}*v[i]*v[j]*{\order[1]{\xi}}[i]*{\order[2]{\xi}}[j]/2 + \dot{\E}_2*\eta**4*{\order[1]{\lambda}}**2*v[i]*w[j, \lambda]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j] + \ddot{\E}_2*\eta**4*{\order[1]{\lambda}}**2*v[i]*v[j]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]/4 + \E_4*\eta**4*v[i]*v[j]*v[k]*v[l]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/24 + \E_3*\eta**3*v[i]*v[j]*v[k]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + \E_3*\eta**4*v[i]*v[j]*w[k, l]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]

The terms of order 0, 1 and 2 in $\eta$ are identically null:

In [10]:
assert ΔE.coeff(η, 0) == 0
assert ΔE.coeff(η, 1) == 0
assert ΔE.coeff(η, 2) == 0

In [28]:
ΔE.coeff(η, 3)

\E_3*v[i]*v[j]*v[k]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/6 + \dot{\E}_2*{\order[1]{\lambda}}*v[i]*v[j]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]/2

In [11]:
ΔE3 = ΔE.coeff(η, 3)

for r in (r"\dot{E}_ij", r"E_ijk", "BifEq1"):
    ΔE3 = ΔE3.subs(rules[r])

In [12]:
#| code-fold: true
display_latex_equation(r"\order[3]{\Delta\E}", ΔE3)

<IPython.core.display.Math object>

Finally, the fourth-order term is simplified.

In [26]:
ΔE4 = ΔE.coeff(η, 4)

#"E₃(v_i, v_j, w_kλ)", r"\dot{E}₂(v_i, w_jλ)",

rules[r"\dot{E}₂(v_i, w_jλ)"] = {
    E2_dot * v_[i] * w_[j, _λ]: (E_ddot_[i, j] - E2_ddot * v_[i] * v_[j]) / 2,
    E2_dot * v_[i] * w_[k, _λ]: (E_ddot_[i, k] - E2_ddot * v_[i] * v_[k]) / 2,
    E2_dot * v_[i] * w_[l, _λ]: (E_ddot_[i, l] - E2_ddot * v_[i] * v_[l]) / 2,
}

rules[r"E₂(w_ij, w_kl)"] = {
    E2 * w_[i, j] * w_[k, l]: - E3 * v_[i] * v_[j] * w_[k, l],
    E2 * w_[i, k] * w_[j, l]: - E3 * v_[i] * v_[k] * w_[j, l],
    E2 * w_[i, l] * w_[j, k]: - E3 * v_[i] * v_[l] * w_[j, k],
}

rules[r"E₂(w_ij, w_kλ)"] = {
    E2 * w_[i, j] * w_[k, _λ]: E2_dot * w_[i, j] * v_[k]
}

rules[r"E₂(w_iλ, w_jλ)"] = {
    E2 * w_[i, _λ] * w_[j, _λ]: -E2_dot * v_[i] * w_[j, _λ],
    E2 * w_[i, _λ] * w_[k, _λ]: -E2_dot * v_[i] * w_[k, _λ],
    E2 * w_[i, _λ] * w_[l, _λ]: -E2_dot * v_[i] * w_[l, _λ],
    E2 * w_[j, _λ] * w_[k, _λ]: -E2_dot * v_[j] * w_[k, _λ],
    E2 * w_[j, _λ] * w_[l, _λ]: -E2_dot * v_[j] * w_[l, _λ],
    E2 * w_[k, _λ] * w_[l, _λ]: -E2_dot * v_[k] * w_[l, _λ],
}
      
rules[r"E₂(w_ij, w_kλ)"] = {
    E2 * w_[i, j] * w_[k, _λ]: E2_dot * w_[i, j] * v_[k],
    E2 * w_[i, j] * w_[l, _λ]: E2_dot * w_[i, j] * v_[l],
    E2 * w_[i, k] * w_[l, _λ]: E2_dot * w_[i, k] * v_[l],
    E2 * w_[i, k] * w_[j, _λ]: E2_dot * w_[i, k] * v_[j],
    E2 * w_[i, l] * w_[j, _λ]: E2_dot * w_[i, l] * v_[j],
    E2 * w_[i, l] * w_[k, _λ]: E2_dot * w_[i, l] * v_[k],
    E2 * w_[j, k] * w_[l, _λ]: E2_dot * w_[j, k] * v_[l],
    E2 * w_[j, k] * w_[i, _λ]: E2_dot * w_[j, k] * v_[i],
    E2 * w_[j, l] * w_[i, _λ]: E2_dot * w_[j, l] * v_[i],
    E2 * w_[j, l] * w_[k, _λ]: E2_dot * w_[j, l] * v_[k],
    E2 * w_[k, l] * w_[i, _λ]: E2_dot * w_[k, l] * v_[i],
    E2 * w_[k, l] * w_[j, _λ]: E2_dot * w_[k, l] * v_[j],
}

rule_names = [
    r"E₂(w_ij, w_kλ)",
    r"E₂(w_iλ, w_jλ)",
    r"E₂(w_ij, w_kl)",
    r"\dot{E}₂(v_i, w_jλ)", 
]

for r in rule_names:
    ΔE4 = ΔE4.subs(rules[r]).expand()

ΔE4

\E_3*{\order[1]{\lambda}}*v[i]*v[j]*w[k, \lambda]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/2 + \E_3*v[i]*v[j]*v[k]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[2]{\xi}}[k]/4 + \E_3*v[i]*v[j]*w[k, l]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/8 + \E_4*v[i]*v[j]*v[k]*v[l]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/24 - \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[j]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]/4 + \ddot{\E}_2*{\order[1]{\lambda}}**2*v[i]*v[k]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[k]/4 + \dot{\E}_2*{\order[1]{\lambda}}*v[i]*v[j]*{\order[1]{\xi}}[i]*{\order[2]{\xi}}[j]/2 + \dot{\E}_2*{\order[1]{\lambda}}*v[i]*w[j, k]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}[k]/2 + \dot{\E}_2*{\order[1]{\lambda}}*v[i]*w[k, l]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[k]*{\order[1]{\xi}}[l]/4 + \dot{\E}_2*{\order[1]{\lambda}}*v[k]*w[i, j]*{\order[1]{\xi}}[i]*{\order[1]{\xi}}[j]*{\order[1]{\xi}}

In [18]:
aux = ΔE.coeff(η, 4).coeff(ξ1_[i]).coeff(ξ1_[j]).coeff(ξ1_[k]).coeff(ξ1_[l]).subs(rules[r"E₂(w_ij, w_kl)"])
aux

\E_3*v[i]*v[j]*w[k, l]/8 + \E_4*v[i]*v[j]*v[k]*v[l]/24

Upon simplification of $\E_2(v_i, \bullet) = 0$, the above expression should be understood as follows

$$
\begin{aligned}[b]
\order[4]{\Delta\E} ={}& \tfrac{1}{8} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_2(w_{ij}, w_{kl})
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \E_2(w_{ij}, w_{k\lambda})\\
&+ \tfrac{1}{2} \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_2(w_{i\lambda}, w_{j\lambda})
+ \tfrac{1}{4} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[2]{\xi}_k \, \E_3(v_i, v_j, v_k)\\
&+ \tfrac{1}{4} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_3(v_i, v_j, w_{kl})
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \E_3(v_i, v_j, w_{k\lambda})\\
&+ \tfrac{1}{24} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \E_4(v_i, v_j, v_k, v_l)
+ \tfrac{1}{4} \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \ddot{\E}_2(v_i, v_j)\\
&+ \tfrac{1}{4} \order[2]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, v_j)
+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[2]{\xi}_j \, \dot{\E}_2(v_i, v_j)\\
&+ \tfrac{1}{2} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \dot{E}_2(v_i, w_{jk})
+ \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, w_{j\lambda})\\
&+ \tfrac{1}{6} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \dot{\E}_3(v_i, v_j, v_k),
\end{aligned}
$$

and, upon reordering

$$
\begin{aligned}
\order[4]{\Delta\E} ={} & \tfrac{1}{24} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \bigl\{ \E_4(v_i, v_j, v_k, v_l) + 3\E_3(v_i, v_j, w_{kl}) + 3\bigl[\E_2(w_{ij}, w_{kl}) + \E_3(v_i, v_j, w_{kl}) \bigr] \bigr\}\\
&+ \tfrac{1}{6} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \bigl\{ \dot{\E}_3(v_i, v_j, v_k) + 3\dot{\E}_2(v_i, w_{jk}) + 3 \bigl[ \E_2(w_{ij}, w_{k\lambda}) + \E_3(v_i, v_j, w_{k\lambda}) \bigr] \bigr\}\\
&+ \tfrac{1}{4} \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \bigl\{ \ddot{\E}_2(v_i, v_j) + 2\dot{\E}_2(v_i, w_{j\lambda}) + 2\bigl[ \E_2(w_{i\lambda}, w_{j\lambda}) + \dot{\E}_2(v_i, w_{j\lambda}) \bigr] \bigr\}\\
&+ \tfrac{1}{4} \order[1]{\xi}_i \, \order[2]{\xi}_j \, \bigl[ \E_3(v_i, v_j, v_k) \, \order[1]{\xi}_k + 2\order[1]{\lambda} \, \dot{\E}_2(v_i, v_j) \bigl]\\
&+ \tfrac{1}{4} \order[2]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, v_j).
\end{aligned}
$$

Testing Eq. (@eq-20230107180410) with $w_{kl}$ and $w_{k\lambda}$, successively and Eq. (@eq-20230107180501) with $w_{j\lambda}$ and further using the first bifurcation equation (@eq-20230125062336)

$$
\begin{aligned}
\order[4]{\Delta\E} ={} & \tfrac{1}{24} \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l \, \bigl[ \E_4(v_i, v_j, v_k, v_l) + 3\E_3(v_i, v_j, w_{kl}) \bigr]\\
&+ \tfrac{1}{6} \order[1]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k  \, \bigl[ \dot{\E}_3(v_i, v_j, v_k) + 3\dot{\E}_2(v_i, w_{jk}) \bigr]\\
&+ \tfrac{1}{4} \bigl( \order[1]{\lambda} \bigr)^2 \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \bigl[\ddot{\E}_2(v_i, v_j) + 2\dot{\E}_2(v_i, w_{j\lambda}) \bigr]\\
&+ \tfrac{1}{4} \order[2]{\lambda} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \dot{\E}_2(v_i, v_j).
\end{aligned}
$$

Now, recognizing the definitions (@eq-20230125062505), (@eq-20230124211207), (@eq-20230124210649) and (@eq-20230124210049) finally delivers

$$
\begin{aligned}[b]
\order[4]{\Delta\E} ={} & \tfrac{1}{24} \bigl\{ E_{ijkl} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k \, \order[1]{\xi}_l + 4\order[1]{\lambda} \, \dot{E}_{ijk} \, \order[1]{\xi}_i \, \order[1]{\xi}_j \, \order[1]{\xi}_k\\
&+ 6 \bigl[ \bigl( \order[1]{\lambda} \bigr)^2 \, \ddot{E}_{ij} + \order[2]{\lambda} \, \dot{E}_{ij} \bigr] \, \order[1]{\xi}_i \, \order[1]{\xi}_j \bigr\}.
\end{aligned}
$$ {#eq-20230402101651}

and the asymptotic expansion (@eq-20230528190539) is retrieved.

## Asymptotic expansion of the hessian of the energy

We use the same code to derive an asymptotic expansion of the hessian $\E_{,uu}$ of the energy along the bifurcated branch.

In [ ]:
E_uu = (eval(data["E_uu"]).subs({Symbol("\lambda"): λ, Symbol("u"): u})  + O(η**3)).expand()

In [ ]:
#| code-fold: true
lsk.display_latex_equation(r"\E_{,uu}[u(\eta), \lambda(\eta)]", E_uu)

The above expression must be understood as follows

$$
\begin{aligned}[b]
\E_{,uu}[u(\eta), \lambda(\eta); \bullet, \bullet] ={} 
& \E_2(\bullet, \bullet) + \eta \, \bigl[ \order[1]{\xi}_i \, \E_3(v_i, \bullet, \bullet) + \order[1]{\lambda} \, \dot{\E}_2(\bullet, \bullet) \bigr]\\
&+ \tfrac{1}{2}\eta^2 \, \bigl[ \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_4(v_i, v_j, \bullet, \bullet)
+ \order[2]{\xi}_i \, \E_3(v_i, \bullet, \bullet) \\
&+ \order[1]{\xi}_i \, \order[1]{\xi}_j \, \E_3(w_{ij}, \bullet, \bullet)
+ 2\order[1]{\lambda} \, \order[1]{\xi}_i \, \E_3(w_{i\lambda}, \bullet, \bullet)\\
&+ 2\order[1]{\lambda} \, \order[1]{\xi}_i \, \dot{\E}_3(v_i, \bullet, \bullet)
+ (\order[1]{\lambda})^2 \, \ddot{\E}_2(\bullet, \bullet) + \order[2]{\lambda} \, \dot{\E}_2(\bullet, \bullet)\bigr],
\end{aligned}
$$

or, equivalently

$$
\begin{aligned}[b]
\E_{,uu}[u(\eta), \lambda(\eta); \bullet, \bullet] ={} 
& \E_2(\bullet, \bullet) + \eta \, \bigl[ \E_3(\order[1]{u}, \bullet, \bullet) + \order[1]{\lambda} \, \dot{\E}_2(\bullet, \bullet) \bigr]\\
&+ \tfrac{1}{2}\eta^2 \, \bigl[ \E_4(\order[1]{u}, \order[1]{u}, \bullet, \bullet)
+ \E_3(\order[2]{u}, \bullet, \bullet) \\
&+ 2\order[1]{\lambda} \, \dot{\E}_3(\order[1]{u}, \bullet, \bullet)
+ (\order[1]{\lambda})^2 \, \ddot{\E}_2(\bullet, \bullet) + \order[2]{\lambda} \, \dot{\E}_2(\bullet, \bullet)\bigr],
\end{aligned}
$$ {#eq-20230517061452}

where $\order[1]{u}$ and $\order[2]{u}$ are given by Eq. (@eq-20230411141319).